In [1]:
import geopandas as gpd

In [10]:
gdf = gpd.read_file('../train_buffered_points.shp')
gdf.head()

,canopy,col,label,row,tile_name,x,y,geometry
0,0.0,572,0,1481,029011,1801590.0,1620360.0,POLYGON ((-75.70782939948658 35.89756989233159...
1,0.0,4608,0,1271,028011,1772670.0,1626660.0,POLYGON ((-76.00927205065202 36.00689937966732...
2,0.0,1060,0,2695,029011,1816230.0,1583940.0,POLYGON ((-75.63450100570053 35.55192775717828...
3,0.0,198,0,3792,029011,1790370.0,1551030.0,"POLYGON ((-75.99243602489075 35.3132232234113,..."
4,0.0,3239,0,500,028012,1731600.0,1499790.0,"POLYGON ((-76.7454097877189 34.97277180541865,..."


In [11]:
gdf = gpd.read_file('../train_buffered_points.shp')
all_pix = []
for index, row in gdf.iterrows():
    all_pix.append(((row['col'], row['row']), row['tile_name']))
len(all_pix)

9000

In [4]:
def tile_generator(self, pixel_info, batch_size, flatten=False, canopy=False):
    ### this is a keras compatible data generator which generates data and labels on the fly 
    ### from a set of pixel locations, a list of image datasets, and a label dataset
    bad_tiles = 0
    tile_size = self.tile_length
    i = 0
    # assuming all images have the same num of bands
    band_count = self.l8_dict['028012'][0].count - 1
    time_steps = len(self.l8_dict['028012'])
    class_count = self.class_count
    buffer = math.floor(tile_size / 2)
    while True:
        lc_batch = np.zeros((batch_size, class_count))
        canopy_batch = np.zeros((batch_size, 1))
        image_batch = np.zeros((batch_size, time_steps, tile_size, tile_size, band_count))
        rnn_image_batch = np.zeros((batch_size, time_steps, band_count))
        b = 0
        while b < batch_size:
            # if we're at the end  of the data just restart
            if i >= len(pixel_locations):
                i=0
            r, c = pixel_info["row"][i], pixel_info["col"][i] ####
            tile_num = pixel_info["tile_name"][i] #####
            i += 1
            tiles_to_read = self.l8_dict[tile_num]
            tiles_read = util.read_windows(tiles_to_read, c ,r, buffer, tile_size)
            rnn_tiles_read = util.read_windows(tiles_to_read, c ,r, 0, 1)
            reshaped_tiles = []
            rnn_reshaped_tiles = []
            band_avg = [  345.72448081,   352.93755735,   319.34257128,   899.39728239,
         649.46125258,   370.53562465, -1084.8218946 ]
            band_std = [ 661.75737932,  363.32761072,  425.28671553,  959.63442896,
        838.86193201,  501.96992987, 3562.42995552]
            for index in range(len(tiles_read)):
                rnn_tile = rnn_tiles_read[index][0:7]
                tile = tiles_read[index][0:7]
                reshaped_tile = reshape_as_image(tile).astype(np.float64)
                rnn_reshaped_tile = reshape_as_image(rnn_tile).astype(np.float64)
                rnn_reshaped_tile = np.divide(np.subtract(rnn_reshaped_tile,band_avg),band_std)                 
                reshaped_tile= np.divide(np.subtract(reshaped_tile, band_avg),band_std)
                reshaped_tiles.append(reshaped_tile)
                rnn_reshaped_tiles.append(rnn_reshaped_tile)
            lc_data = pixel_info["label"][i]
            canopy_data = pixel_info["label"][i]
            lc_label = self.one_hot_encode(lc_data, 1, class_count)
            lc_batch[b] = lc_label.reshape(class_count)
            canopy_batch[b] = canopy_data.reshape(1) / 100
            rnn_total_tile = np.array((*rnn_reshaped_tiles,))
            rnn_image_batch[b] = rnn_total_tile.reshape((len(tiles_read),7))
            total_tile = np.array((*reshaped_tiles,))
            image_batch[b] = total_tile
            b += 1
            if canopy:
                yield ({"rnn_input":rnn_image_batch, "tile_input":image_batch}, {'landcover': lc_batch, 'canopy': canopy_batch})
            else: 
                yield (image_batch, lc_batch)

In [10]:
train = gpd.read_file('train_buffered_points.shp')